In [1]:
from astropy.io import fits
from astropy import units as u
from astropy.table import Table, Column, MaskedColumn
from astropy.time import Time
from astropy.coordinates import SkyCoord, Distance
from astroquery.vizier import Vizier
import numpy as np

In [2]:
def gaiadr3_query(
    ra: list,
    dec: list,
    rad: float = 1.0,
    maxmag: float = 25,
    maxsources: float = 1000000,
):
    """
    Acquire the Gaia DR3.

    Uses astroquery.vizier to query the Gaia Data Release 3 (DR3) catalog based on given right ascension (RA), declination (Dec), and other criteria.

    Parameters
    ----------
    ra : list
        RA of center in degrees.
    dec : list
        Dec of center in degrees.
    rad : float
        Search radius in degrees.
    maxmag : float
        Maximum magnitude limit.
    maxsources : float
        Maximum number of sources to return.

    Returns
    -------
    Table
        table of reference catalog.

    Examples
    --------
    >>> catalog = gaiadr3_query(ra, dec, rad, maxmag, maxsources)
    """
    # Astrometry-related parameters and constraints
    vquery = Vizier(
        columns=["RA_ICRS", "DE_ICRS", "pmRA", "pmDE", "Plx", "RVDR2", "Gmag"],
        row_limit=maxsources,
        column_filters={"Gmag": ("<%f" % maxmag), "Plx": ">0"},
    )

    # The GaiaDR3 sources in the sky field
    coord = SkyCoord(ra=ra, dec=dec, unit=u.deg, frame="icrs")

    # The GaiaDR3 sources in the sky field
    result = vquery.query_region(coord, radius=rad * u.deg, catalog="I/355/gaiadr3")
    table = result[0]

    return table

## Image

In [3]:
img_path = "\\nfsdata\share\pipeline-unittest\csst_mci\csst_mci_instrument\CSST_MCI_C1_EXDF_20240419024541_20240419025041_20100000001_07_L1_V01_img.fits".replace("\\", "/")

In [4]:
img_hdul = fits.open(img_path)

In [5]:
img_hdul

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fa180061150>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7fa1a806ec10>]

In [6]:
img_hdul[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
NEXTEND =                    1 / number of array dimensions                     
DATE    = '2024-04-19T03:57:42.8' / written date (yyyy-mm-ddThh:mm:ss.s)        
FILENAME= 'CSST_MCI_C1_EXDF_20240419024541_20240419025041_20100000001_07_L0_V01'
FILETYPE= 'EXDF    '           / observation type                               
TELESCOP= 'CSST    '           / telescope name                                 
INSTRUME= 'MCI     '           / instrument name                                
RADECSYS= 'ICRS    '           / coordinate system of the object                
EQUINOX =               2000.0                                                  
FITSSWV = 'mci_sim_0.8.03'  

In [7]:
img_hdul[0].data

In [8]:
img_hdul[1].header

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 9216                                                  
NAXIS2  =                 9232                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
EXTNAME = 'SCI     '                                                            
EXTVER  =                    1                                                  
BUNIT   = 'e/s     '           / physical unit of array values                  
COMMENT ========================================================================
COMMENT FILE INFORMATION                                                        
COMMENT ====================

In [9]:
img_hdul[1].data

array([[2.8623958, 3.2563713, 5.7914863, ..., 5.089698 , 2.245794 ,
        5.6442103],
       [3.8560715, 4.5986705, 3.9977348, ..., 3.7632573, 3.3575957,
        4.6310725],
       [2.458934 , 4.496589 , 4.8904715, ..., 4.168032 , 6.0553613,
        2.1957495],
       ...,
       [4.1140184, 5.240286 , 3.364142 , ..., 4.1308312, 4.7383265,
        3.1674178],
       [4.0551357, 3.8175957, 3.4682097, ..., 3.2136004, 2.0385983,
        4.587079 ],
       [2.3401625, 2.9723327, 4.0057793, ..., 4.539656 , 2.7061558,
        2.303656 ]], dtype='>f4')

In [10]:
image_header = fits.getheader(img_path, ext=0)
pointing_ra = image_header["RA_OBJ"]
pointing_dec = image_header["DEC_OBJ"]
refcat = gaiadr3_query(pointing_ra, pointing_dec, rad=2.0)

In [64]:
type(refcat)

astropy.table.table.Table

In [12]:
refcat.remove_row(11443)

In [208]:
type(refcat["DE_ICRS"])

astropy.table.column.MaskedColumn

In [223]:
np.abs(refcat["Plx"]).quantity

<Quantity [2.2596, 0.923 , 1.7342, ..., 0.7872, 0.2548, 1.5644]>

## Reference catalog

In [215]:
ref_path = "\\nfsdata\share\pipeline-unittest\csst_mci\csst_mci_astrometry\l1_proc_output\gaiadr3.fits".replace("\\", "/")

In [216]:
ref_hudl = fits.open(ref_path)

In [159]:
catalog = Table(ref_hudl[1].data)
# ra = catalog["RA_ICRS"] * u.deg
# dec = catalog["DE_ICRS"] * u.deg
# parallax = catalog["Plx"] * u.mas
# pmra = catalog["pmRA"] * u.mas
# pmdec = catalog["pmDE"] * u.mas

ra = Column(name="RA_ICRS", data=catalog["RA_ICRS"], unit=u.deg)
dec = Column(name="DE_ICRS", data=catalog["DE_ICRS"], unit=u.deg)
parallax = Column(name="Plx", data=catalog["Plx"], unit=u.mas)
parallax = refcat["Plx"]
# parallax = Column(name="Plx", data=catalog["Plx"], unit=u.mas)
pmra = Column(name="pmRA", data=catalog["pmRA"], unit=u.mas / u.yr)
pmdec = Column(name="pmDE", data=catalog["pmDE"], unit=u.mas / u.yr)
# catalog = refcat
# ra = catalog["RA_ICRS"]
# dec = catalog["DE_ICRS"]
# parallax = catalog["Plx"]
# pmra = catalog["pmRA"]
# pmdec = catalog["pmDE"]
# mag = catalog["Gmag"]
cut = (abs(pmra) > 0) & (abs(pmdec) > 0)
c = SkyCoord(
    ra=ra[cut],
    dec=dec[cut],
    distance=Distance(parallax=np.abs(parallax[cut]) * u.mas),
    pm_ra_cosdec=pmra[cut],
    pm_dec=pmdec[cut],
    obstime=Time(2016.0, format="jyear", scale="utc"),
    frame="icrs",
)

# Set the reference epoch to 2000
epochobs = Time(2000.0, format="jyear", scale="utc")
# Compute the celestial positions at the epoch
c_epoch_now = c.apply_space_motion(epochobs)

/home/fangwf/software/miniconda3/envs/csst_mci/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 138 of "distance overridden (Note 6)", 1 of "binary logical OR of the above warnings"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [160]:
# parallax = abs(refcat["Plx"][cut]) * u.mas
parallax = np.abs(Column(name="Plx", data=catalog["Plx"], unit=u.mas, format='{:9.4f}', description='? Parallax (parallax)')).quantity

In [199]:
type(Column(name="Plx", data=catalog["Plx"], unit=u.mas))

astropy.table.column.Column

In [162]:
ref_hudl[0]

In [164]:
ref_hudl[1].data

FITS_rec([(116.4613841 , 37.43569588, -1.152,  -4.863, 2.2596, 20.808371),
          (116.28728873, 37.42637767, -7.179,  -2.302, 0.923 , 19.755737),
          (116.26514877, 37.42503616, -6.796, -11.013, 1.7342, 17.05047 ),
          ...,
          (114.57579799, 40.98774444,  0.781,  -8.316, 0.7872, 14.156981),
          (114.56369327, 40.99372598,  0.334,  -0.444, 0.2548, 17.488308),
          (114.59090475, 40.99928735, 25.997, -35.978, 1.5644, 18.949257)],
         dtype=(numpy.record, [('RA_ICRS', '>f8'), ('DE_ICRS', '>f8'), ('pmRA', '>f8'), ('pmDE', '>f8'), ('Plx', '>f8'), ('Gmag', '>f8')]))

In [201]:
dcol = fits.ColDefs(ref_hudl[1].data)
type(dcol)

astropy.io.fits.column.ColDefs

In [178]:
tbl2 = fits.BinTableHDU.from_columns(dcol)
tbl2.data

FITS_rec([(116.4613841 , 37.43569588, -1.152,  -4.863, 2.2596, 20.808371),
          (116.28728873, 37.42637767, -7.179,  -2.302, 0.923 , 19.755737),
          (116.26514877, 37.42503616, -6.796, -11.013, 1.7342, 17.05047 ),
          ...,
          (114.57579799, 40.98774444,  0.781,  -8.316, 0.7872, 14.156981),
          (114.56369327, 40.99372598,  0.334,  -0.444, 0.2548, 17.488308),
          (114.59090475, 40.99928735, 25.997, -35.978, 1.5644, 18.949257)],
         dtype=(numpy.record, [('RA_ICRS', '<f8'), ('DE_ICRS', '<f8'), ('pmRA', '<f8'), ('pmDE', '<f8'), ('Plx', '<f8'), ('Gmag', '<f8')]))

In [193]:
tbl2 = fits.BinTableHDU.from_columns(ref_hudl[1].data)
tbl2.data

FITS_rec([(116.4613841 , 37.43569588, -1.152,  -4.863, 2.2596, 20.808371),
          (116.28728873, 37.42637767, -7.179,  -2.302, 0.923 , 19.755737),
          (116.26514877, 37.42503616, -6.796, -11.013, 1.7342, 17.05047 ),
          ...,
          (114.57579799, 40.98774444,  0.781,  -8.316, 0.7872, 14.156981),
          (114.56369327, 40.99372598,  0.334,  -0.444, 0.2548, 17.488308),
          (114.59090475, 40.99928735, 25.997, -35.978, 1.5644, 18.949257)],
         dtype=(numpy.record, [('RA_ICRS', '<f8'), ('DE_ICRS', '<f8'), ('pmRA', '<f8'), ('pmDE', '<f8'), ('Plx', '<f8'), ('Gmag', '<f8')]))

In [179]:
tblhdr = np.array([ref_hudl[1].header.tostring()])
# Define the column properties and create a new table HDU for the header information
hcol = fits.ColDefs([fits.Column(name="Field Header Card", array=tblhdr, format="13200A")])
tbl1 = fits.BinTableHDU.from_columns(hcol)

In [197]:
type(fits.Column(name="Field Header Card", array=tblhdr, format="13200A"))

astropy.io.fits.column.Column

In [200]:
type(Column(name="Plx", data=catalog["Plx"], unit=u.mas))

astropy.table.column.Column

In [190]:
tblhdr = np.array([ref_hudl[1].header.tostring()])
# Define the column properties and create a new table HDU for the header information
tbl1 = fits.BinTableHDU.from_columns([fits.Column(name="Field Header Card", array=tblhdr, format="13200A")])